<a href="https://colab.research.google.com/github/KarthikGowdaRamakrishna/Adv-Techniques-With-LLM-INFO7374-Spring-2025/blob/main/INFO_7374_Assignment_3_Karthik_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3  
**100 points**  

## Task  
Implement the *Self Alignment with Instruction Backtranslation* paper. When fine-tuning the model, use **LoRA**. You will not be able to do full fine-tuning due to memory constraints.  

📄 **Paper Link:** [Self Alignment with Instruction Backtranslation](https://arxiv.org/pdf/2308.06259.pdf)  

💡 **Colab’s GPU usage is limited.** First, prototype on **CPU** before training on **GPU** with the full dataset. If GPU access on Colab is restricted, try **PyTorch Lightning Studio** or **Kaggle notebooks**.  

---

## Instructions  

### 1. Fine-tune the Backward Model (25 points)  
- Fine-tune the base language model (**LLaMA 2 7B**) using (output, instruction) pairs **{(yi, xi)}** from the **seed data** to obtain a **backward model**:  
  $$M_{yx} := p(x|y)$$  
- This model should predict the **instruction** from the given **output**.  
- Use the **OpenAssistant-Guanaco** training dataset.  

📌 **Deliverable:** Push the **backward model** to Hugging Face and paste the URL here.  

---

### 2. Self-Augmentation (25 points)  
- Randomly sample **150** examples and generate instructions from the **LIMA dataset's completions**.  
- **Filter out multi-turn examples.**  
- **Print 5 examples** of the generated instructions.  

**Example:**  
✅ **Single-turn:**  
- *(What is the capital of France?, Paris)*  
❌ **Multi-turn:**  
- *(What is the meaning of life? → 42 → Why is it 42? → That’s the universe...)*  

📌 **Deliverable:** Print **5 generated instructions**.  

---

### 3. Self-Curation (25 points)  
- Use **few-shot prompting** and **Table 1** from the paper to select **high-quality examples**.  
- Filter out **bad samples** using an **LLM rating system**.  
- Use **LLaMA-7B-Chat-HF** to evaluate:  
  ```plaintext
  LLM(“Evaluate the quality of the instruction/response pair” + example. Rate it from 1-5)


In [ ]:
!pip install transformers accelerate


In [ ]:
from huggingface_hub import login

login(token="")  # Replace with your actual token
login()



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
from datasets import load_dataset

# ✅ Step 1: Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Required for padding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:


# ✅ Step 2: Quantization + LoRA config (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

# ✅ Step 3: Load 4-bit model and prepare for LoRA
model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    device_map="auto",
    torch_dtype=torch.float16  # or bfloat16 if using A100
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
inputs

{'input_ids': tensor([[   1, 2296,  338]]), 'attention_mask': tensor([[1, 1, 1]])}

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=5)

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2208: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
outputs


tensor([[   1, 2296,  338,  263, 8750,  310,  263, 1887]])

In [ ]:
res = tokenizer.decode(outputs[0])

In [ ]:
res

'<s> She is a daughter of a local'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
from datasets import load_dataset

# Step 1: Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Required for padding

# Step 2: Quantization + LoRA config (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

# Step 3: Load 4-bit model and prepare for LoRA
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# Step 4: Dataset preparation (BACKWARD MODEL => p(x|y))
dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

def flip_and_tokenize(example):
    try:
        full_text = example["text"]
        if "### Assistant:" not in full_text:
            raise ValueError("No assistant response found.")
        instruction = full_text.split("### Assistant:")[0].replace("### Human:", "").strip()
        response = full_text.split("### Assistant:")[1].strip()
    except Exception:
        instruction, response = "", ""
    prompt = f"### Response:\n{response}\n\n### Instruction:\n{instruction}"
    tokens = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Use small subset to avoid OOM in Colab
sampled_dataset = dataset.shuffle(seed=42).select(range(2000))
tokenized_dataset = sampled_dataset.map(flip_and_tokenize, remove_columns=["text"])
tokenized_dataset.set_format("torch")

# Step 5: Training arguments
training_args = TrainingArguments(
    output_dir="./llama2-backward-model",
    per_device_train_batch_size=1,         # T4 has ~16GB VRAM
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=True,                             # T4 supports fp16
    bf16=False,                            # No bf16 on T4
    remove_unused_columns=False
)

# Step 6: Custom Trainer
class MyTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits", outputs[0])
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Step 7: Train
trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<ipython-input-12-5543a59dabbc>:86: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MyTrainer.__init__`. Use `processing_class` instead.
  trainer = MyTrainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*a

Step,Training Loss
10,56.276400
20,14.157600
30,3.630300
40,3.168500
50,3.668100
60,3.234200
70,2.934100
80,3.418600
90,3.119000
100,2.803100


Step,Training Loss
10,56.276400
20,14.157600
30,3.630300
40,3.168500
50,3.668100
60,3.234200
70,2.934100
80,3.418600
90,3.119000
100,2.803100


TrainOutput(global_step=500, training_loss=4.275940151214599, metrics={'train_runtime': 4535.9808, 'train_samples_per_second': 0.441, 'train_steps_per_second': 0.11, 'total_flos': 4.062128898048e+16, 'train_loss': 4.275940151214599, 'epoch': 1.0})

In [ ]:
model.push_to_hub("KarthikrGowda/llama2-backward-lora-fineTuned")
tokenizer.push_to_hub("KarthikrGowda/llama2-backward-lora-fineTuned")


adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KarthikrGowda/llama2-backward-lora-fineTuned/commit/65fa4460f4a2e4cebfaa86dcc5faed87878ac51f', commit_message='Upload tokenizer', commit_description='', oid='65fa4460f4a2e4cebfaa86dcc5faed87878ac51f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KarthikrGowda/llama2-backward-lora-fineTuned', endpoint='https://huggingface.co', repo_type='model', repo_id='KarthikrGowda/llama2-backward-lora-fineTuned'), pr_revision=None, pr_num=None)

## I wanted print a sample output but my kernel kept crashing

### 2. Self-Augmentation (25 points)  
- Randomly sample **150** examples and generate instructions from the **LIMA dataset's completions**.  
- **Filter out multi-turn examples.**  
- **Print 5 examples** of the generated instructions.  

**Example:**  
✅ **Single-turn:**  
- *(What is the capital of France?, Paris)*  
❌ **Multi-turn:**  
- *(What is the meaning of life? → 42 → Why is it 42? → That’s the universe...)*  

📌 **Deliverable:** Print **5 generated instructions**.  

---


In [ ]:
!pip install datasets

In [ ]:
!pip install datasets transformers accelerate


In [7]:
import random
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer


### STEP 1: Load the LIMA dataset and preview

In [9]:
lima = load_dataset("GAIR/lima", split="train")

# Print out one example to see structure
print("====== Sample from LIMA dataset ======")
print(lima[0])

====== Sample from LIMA dataset ======
{'conversations': ['Can brain cells move? By movement I mean long distance migration (preferably within the brain only).', 'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Mil


### STEP 2: Shuffle and randomly sample 150 examples

### We'll fix a random seed for reproducibility


In [11]:
sampled = lima.shuffle(seed=42).select(range(150))

print("\n====== After sampling 150 examples, here is one example ======")
print(sampled[0])



====== After sampling 150 examples, here is one example ======
{'conversations': ["How do you know if you're in a healthy relationship?", 'It is important to understand that there is no "one size fits all" answer to your question. Every relationship is different, and there is no single way to define a "healthy" relationship.\n\nThat said, there are some general guidelines that you can use. One of the most important things to remember is that a healthy relationship is based on mutual respect. In a healthy relationship, you should feel comfortable being yourself, and you should feel that your partner respects and values you as a person.\n\nAnother important aspect of a healthy relationship is honesty. In a healthy relationship, you should feel comfortable being open and honest with your partner. Your partner should feel the same way towards you. If you find that you are lying to your partner or that your partner is lying to you, that is a sign that the relationship is not healthy.\n\nIn

## STEP 3: Filter out multi-turn conversations (keep only single-turn with 2 msgs)



In [12]:

def is_single_turn(example):
    """
    Returns True if 'conversations' is exactly 2 messages long.
    """
    return (
        isinstance(example["conversations"], list)
        and len(example["conversations"]) == 2
    )
filtered = sampled.filter(is_single_turn)
print(f"\n====== Filter results ======")
print(f"Filtered down to {len(filtered)} single-turn examples.")

if len(filtered) == 0:
    print("No single-turn examples found in this subset!")
    # You might want to handle this case or re-check your data
else:
    print("Here's one single-turn example after filtering:")
    print(filtered[0])


====== Filter results ======
Filtered down to 147 single-turn examples.
Here's one single-turn example after filtering:
{'conversations': ["How do you know if you're in a healthy relationship?", 'It is important to understand that there is no "one size fits all" answer to your question. Every relationship is different, and there is no single way to define a "healthy" relationship.\n\nThat said, there are some general guidelines that you can use. One of the most important things to remember is that a healthy relationship is based on mutual respect. In a healthy relationship, you should feel comfortable being yourself, and you should feel that your partner respects and values you as a person.\n\nAnother important aspect of a healthy relationship is honesty. In a healthy relationship, you should feel comfortable being open and honest with your partner. Your partner should feel the same way towards you. If you find that you are lying to your partner or that your partner is lying to you, t

## STEP 4: Load your backwards model (the model that goes completion→instruction)



In [13]:
backward_model_name = "KarthikrGowda/llama2-backward-lora-fineTuned"
tokenizer = AutoTokenizer.from_pretrained(backward_model_name)
model = AutoModelForCausalLM.from_pretrained(backward_model_name, torch_dtype=torch.float16).to("cuda")
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, b

### STEP 5: Use the backwards model to generate instructions from LIMA completions



In [14]:
def generate_instruction_from_completion(completion):
    """
    We feed the LIMA completion text into the backwards model,
    prompting it to produce the "instruction" that might have prompted this answer.
    """
    # You can customize the prompt as you like, e.g.:
    prompt = (
        "Given the following answer, generate the best possible user instruction.\n\n"
        f"Answer:\n{completion}\n\n"
        "Instruction:"
    )

    # Tokenize and generate
    input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=32,
            temperature=0.7,
            do_sample=True
        )
    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract everything after "Instruction:" as the generated instruction
    if "Instruction:" in full_output:
        instruction_part = full_output.split("Instruction:", 1)[1].strip()
    else:
        instruction_part = full_output.strip()

    return instruction_part


### STEP 6: Construct the (generated-instruction, LIMA-response) pairs


In [15]:
!pip install tqdm


In [ ]:
for idx, ex in enumerate(filtered[:5]):
    print(idx, type(ex), ex if idx < 5 else '')



0 <class 'str'> conversations
1 <class 'str'> source


In [16]:
from tqdm.auto import tqdm  # or simply `from tqdm import tqdm`

pairs = []
for ex in tqdm(filtered, desc="Generating instructions"):
    # The LIMA "completion" is the second message in `conversations`
    lima_completion = ex["conversations"][1]

    # Generate instruction from the backwards model
    generated_instruction = generate_instruction_from_completion(lima_completion)

    # Store your new pair
    pairs.append({
        "generated_instruction": generated_instruction,
        "lima_response": lima_completion
    })
# pairs = []
# for ex in filtered:
#     # The LIMA "completion" is the second message in `conversations`
#     lima_completion = ex["conversations"][1]

#     # Ask the backwards model to generate an instruction from that completion
#     generated_instruction = generate_instruction_from_completion(lima_completion)

#     # Store your new pair
#     pairs.append({
#         "generated_instruction": generated_instruction,
#         "lima_response": lima_completion
#     })

Generating instructions:   0%|          | 0/147 [00:00<?, ?it/s]

################################################################################
# STEP 7: Print out five examples of (instruction, response) pairs
################################################################################


In [17]:
print("\n====== Here are 5 random (instruction, response) pairs =====")
for i, example in enumerate(random.sample(pairs, k=min(5, len(pairs)))):
    print(f"\n--- Example {i+1} ---")
    print("Generated instruction:", example["generated_instruction"])
    print("LIMA response:", example["lima_response"])


====== Here are 5 random (instruction, response) pairs =====

--- Example 1 ---
Generated instruction: You have a set of 10 strings, and need to find the best possible match for each of them. The strings are each 10
LIMA response: Fuzzy String Matching is the process of performing a human-like estimation of the similarity of two words or phrases. In many cases, it involves implementing the Levenshtein distance algorithm, which determines how many changes must be made to a string or phrase to turn it into another string or phrase. The following implementation in VBA involves a weighted comparison of the length of the two phrases, the number of changes between each phrase, and whether each word could be found in the target entry.

```
'Calculate the Levenshtein Distance between two strings (the number of insertions,
'deletions, and substitutions needed to transform the first string into the second)
Public Function LevenshteinDistance(ByRef S1 As String, ByVal S2 As String) As Long
    D

In [ ]:
print("\n====== Here are 5 random (instruction, response) pairs =====")
for i, example in enumerate(random.sample(pairs, k=min(5, len(pairs)))):
    print(f"\n--- Example {i+1} ---")
    print("Generated instruction:", example["generated_instruction"])
    print("LIMA response:", example["lima_response"])


====== Here are 5 random (instruction, response) pairs =====

--- Example 1 ---
Generated instruction: Write a program to implement Dijkstra's single-source shortest path algorithm in C language. The program utilizes an adjacency matrix as
LIMA response: Dijkstra’s algorithm is very similar to Prim’s algorithm for minimum spanning tree. Like Prim’s MST, generate a SPT (shortest path tree) with a given source as a root. Maintain two sets, one set contains vertices included in the shortest-path tree, other set includes vertices not yet included in the shortest-path tree. At every step of the algorithm, find a vertex that is in the other set (set not yet included) and has a minimum distance from the source.

```
// The program aims to implement Dijkstra's single-source shortest path algorithm in C language. 
// The program utilizes an adjacency matrix as the representation of the graph

#include <limits.h>
#include <stdbool.h>
#include <stdio.h>

// The number of vertices present in a gr

### 3. Self-Curation (25 points)  
- Use **few-shot prompting** and **Table 1** from the paper to select **high-quality examples**.  
- Filter out **bad samples** using an **LLM rating system**.  
- Use **LLaMA-7B-Chat-HF** to evaluate:  
  ```plaintext
  LLM(“Evaluate the quality of the instruction/response pair” + example. Rate it from 1-5)


## Load Your LLM for Rating

In [ ]:
!pip install --upgrade bitsandbytes


In [1]:
from huggingface_hub import login

login(token="")
login()



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Build a pipeline for text generation
rating_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


## Few-Shot Prompt to Rate Each Pair
We want the LLM to see an (instruction, response) pair, and produce a numeric rating from 1 to 5. You can incorporate the style from the “Self-Alignment with Instruction Backtranslation” paper’s Table 1 prompts, plus a few examples. For instance:

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,      # or load_in_4bit=True
    llm_int8_threshold=6.0
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Now create your pipeline for rating
rating_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [5]:
rating_prompt = """You are a helpful assistant tasked with evaluating instruction/response pairs.
Rate them on a scale of 1 to 5, where:
1 = Very poor quality
2 = Low quality
3 = Average
4 = Good
5 = Excellent

Consider:
- Relevance
- Clarity
- Correctness
- Completeness

Format your answer **ONLY** as a single integer from 1 to 5. No extra text.

Here are some examples:

Example A
Instruction: "What is the capital of Germany?"
Response: "It's Berlin."
Rating: 5

Example B
Instruction: "Tell me how to break into cars."
Response: "Here is how to break the law..."
Rating: 1

Now evaluate this new pair:
Instruction: {instruction}
Response: {response}
Rating:
"""


##Write a Function to Get the Rating

In [ ]:
# Filter out malformed pairs before rating
clean_pairs = [p for p in pairs if "generated_instruction" in p and "lima_response" in p]


In [31]:
def rate_pair(instruction, response):
    """
    Returns an integer rating 1 to 5 based on the LLM's output.
    """
    prompt = rating_prompt.format(instruction=instruction, response=response)

    # Generate from the LLM, limiting tokens
    generation = rating_pipeline(
        prompt,
        max_new_tokens=5,
        temperature=0.0,
        do_sample=False
    )

    # Extract text
    text_output = generation[0]["generated_text"]

    for char in text_output:
        if char in "12345":
            return int(char)

    # If it fails, default to 3 (average)
    return 3


In [32]:
pairs.append({
    "instruction": generated_instruction,
    "response": lima_completion
})


In [33]:
rated_pairs = []
for pair in tqdm(clean_pairs, desc="Rating pairs"):
    instr = pair["generated_instruction"]
    resp = pair["lima_response"]
    rating = rate_pair(instr, resp)

    rated_pairs.append({
        "generated_instruction": instr,
        "lima_response": resp,
        "rating": rating
    })

    torch.cuda.empty_cache()



Rating pairs:   0%|          | 0/147 [00:00<?, ?it/s]

##Filter Out Bad Examples

In [39]:
low_quality = [p for p in rated_pairs if p["rating"] <= 2]
high_quality = [p for p in rated_pairs if p["rating"] >= 4]

print(f"Found {len(low_quality)} low-quality examples")
print(f"Found {len(high_quality)} high-quality examples")


Found 0 low-quality examples
Found 147 high-quality examples


In [36]:
import random

print("\n===== 5 High-Quality Examples =====")
for example in random.sample(high_quality, k=min(5, len(high_quality))):
    print("\nInstruction:", example["generated_instruction"])
    print("Response:", example["lima_response"])
    print("Rating:", example["rating"])

print("\n===== 5 Low-Quality Examples =====")
for example in random.sample(low_quality, k=min(5, len(low_quality))):
    print("\nInstruction:", example["generated_instruction"])
    print("Response:", example["lima_response"])
    print("Rating:", example["rating"])



===== 5 High-Quality Examples =====

===== 5 Low-Quality Examples =====

Instruction: Describe the journey of the human race from a small single cell life-form to the point where we are now, where we can travel to the stars
Response: Once upon a time, Death was the teeth that hunted us under a bright moon. We ran and we shivered, waving our little fire-sticks around.
Once we had settled, Death was the dry season that failed to end, the diseases that killed entire villages. She was famine and boils and warfare.
We grew a little bit and started thinking too much. Death became the emptiness behind words on paper, extinction of languages and passenger pigeons. Death became forgetfulness.
Soon, we no longer needed to have five children to ensure that two would survive to adulthood. We conquered disease, destroyed warfare, filled our souls to the brim with knowing and still didn't understand anything.
We stand on the surface of the moon, or mars, or alpha centauri, and we look back at the s

##  Push to HF Hub

In [40]:
if len(high_quality) == 0:
    print("No high-quality examples found.")
else:
    ds_high = Dataset.from_list(high_quality)
    ...

if len(low_quality) == 0:
    print("No low-quality examples found.")
else:
    ds_low = Dataset.from_list(low_quality)
    ...


No low-quality examples found.


In [42]:
from datasets import Dataset, DatasetDict

ds_all = Dataset.from_list(rated_pairs)

ds_high = Dataset.from_list(high_quality)

ds_dict_contents = {
    "all": ds_all,
    "high_quality": ds_high
}

if len(low_quality) > 0:
    ds_low = Dataset.from_list(low_quality)
    ds_low = ds_low.cast(ds_all.features)
    ds_dict_contents["low_quality"] = ds_low
else:
    print("No low-quality examples found, skipping that split.")

ds_dict = DatasetDict(ds_dict_contents)
ds_dict.push_to_hub("KarthikrGowda/curated-lima-instruction-pairs")


No low-quality examples found, skipping that split.


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/KarthikrGowda/curated-lima-instruction-pairs/commit/1ec11f1db3441691e0009a109cf5b930c3ea45be', commit_message='Upload dataset', commit_description='', oid='1ec11f1db3441691e0009a109cf5b930c3ea45be', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KarthikrGowda/curated-lima-instruction-pairs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KarthikrGowda/curated-lima-instruction-pairs'), pr_revision=None, pr_num=None)

## 4. Fine-tuning on Curated Dataset (25 points)

- Fine-tune the **base model (e.g., LLaMA-2 7B with LoRA)** using the **filtered dataset from Step 3**.
- Print **5 example responses** generated from the fine-tuned model.



In [ ]:
!pip install --upgrade transformers accelerate bitsandbytes datasets tqdm peft


##2. Imports and Hugging Face Login

In [1]:
import torch
import os
from huggingface_hub import notebook_login
notebook_login()  # Paste your Hugging Face token when prompted

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset


###3. Gather Your Curated Dataset

In [18]:

from huggingface_hub import hf_hub_download
from datasets import load_dataset

# Download the `high_quality` parquet file from your dataset
file_path = hf_hub_download(
    repo_id="KarthikrGowda/curated-lima-instruction-pairs",
    filename="data/high_quality-00000-of-00001.parquet",  # Adjust if your file name differs
    repo_type="dataset"
)

ds = load_dataset(
    "parquet",
    data_files={"train": file_path},
    split="train"
)
print(ds)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['generated_instruction', 'lima_response', 'rating'],
    num_rows: 147
})


### 4. Convert Each Row to (input, output) Pairs

In [19]:
train_pairs = []
for item in ds:
    train_pairs.append({
        "input": item["generated_instruction"],
        "output": item["lima_response"]
    })

print("Number of training pairs:", len(train_pairs))
print("Sample pair:", train_pairs[0])


Number of training pairs: 147
Sample pair: {'input': 'Write a user instruction that gives the above answer in a way that is easy to understand.\n\n\n### Instruction:\nWhat is a healthy', 'output': 'It is important to understand that there is no "one size fits all" answer to your question. Every relationship is different, and there is no single way to define a "healthy" relationship.\n\nThat said, there are some general guidelines that you can use. One of the most important things to remember is that a healthy relationship is based on mutual respect. In a healthy relationship, you should feel comfortable being yourself, and you should feel that your partner respects and values you as a person.\n\nAnother important aspect of a healthy relationship is honesty. In a healthy relationship, you should feel comfortable being open and honest with your partner. Your partner should feel the same way towards you. If you find that you are lying to your partner or that your partner is lying to you, 

## 5. Build a Prompt for Each Sample

In [20]:
def format_example(example):
    return {"text": f"### Instruction\n{example['input']}\n\n### Response\n{example['output']}"}

train_dataset = Dataset.from_list(train_pairs)
formatted = train_dataset.map(format_example)


Map:   0%|          | 0/147 [00:00<?, ? examples/s]

##6. Tokenize

In [21]:
model_name = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(ex):
    out = tokenizer(
        ex["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    out["labels"] = out["input_ids"].copy()
    return out

tokenized = formatted.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Map:   0%|          | 0/147 [00:00<?, ? examples/s]

##7. Load Model in 4-bit & Prepare LoRA

In [22]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Prepare for k-bit training
base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(base_model, lora_config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 7. Load Model in 4-bit & Prepare LoRA

In [23]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Prepare for k-bit training
base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(base_model, lora_config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 8. Trainer Setup

In [ ]:
!pip install --upgrade transformers

In [28]:
training_args = TrainingArguments(
    output_dir="./llama2-instruction-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",

    fp16=True,
    report_to="none"
)


def data_collator(features):
    return {
        "input_ids": torch.stack([f["input_ids"] for f in features]),
        "attention_mask": torch.stack([f["attention_mask"] for f in features]),
        "labels": torch.stack([f["labels"] for f in features])
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)


<ipython-input-28-5b704ba7a354>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [29]:
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,8.508800
20,3.414700
30,1.741900


TrainOutput(global_step=36, training_loss=4.034720500310262, metrics={'train_runtime': 58.3463, 'train_samples_per_second': 2.519, 'train_steps_per_second': 0.617, 'total_flos': 2924732806594560.0, 'train_loss': 4.034720500310262, 'epoch': 0.9795918367346939})

In [30]:
import random

model.eval()

def generate_response(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True
        )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("\n===== 5 Example Responses =====")
for i in range(5):
    sample = random.choice(train_pairs)
    user_prompt = sample["input"]
    print(f"\n--- Example {i+1} ---")
    print("Instruction:", user_prompt)
    output_text = generate_response(user_prompt, max_new_tokens=40)
    print("Model output:", output_text)



===== 5 Example Responses =====

--- Example 1 ---
Instruction: ### This is the surface function

```
float sphere(vec3 p)
{
  vec3 q = vec3(0
Model output: ### This is the surface function

```
float sphere(vec3 p)
{
  vec3 q = vec3(0.0);
  float r = 0.0;
  for (int i=0; i<3; i++) {
    q[i] = p[i];

--- Example 2 ---
Instruction: You need to write a script for a short video demonstrating the different ways of flipping someone off.

Script:

"Hi
Model output: You need to write a script for a short video demonstrating the different ways of flipping someone off.

Script:

"Hi there, I'm ____ and I'm here to show you how to flip someone off. Flipping someone off is a great way to show them you don't like them

--- Example 3 ---
Instruction: Please write a story that begins with "Once upon a time, a brave young boy was trapped in a giant tower. The tower was made of metal
Model output: Please write a story that begins with "Once upon a time, a brave young boy was trapped in a giant tower

In [31]:
repo_name = "KarthikrGowda/llama2-instruction-finetuned"  # example
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"Model pushed to: https://huggingface.co/{repo_name}")


adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

✅ Model pushed to: https://huggingface.co/KarthikrGowda/llama2-instruction-finetuned
